## Customer Config
#### DynamoDB Table : wfm-&lt;teamName&gt;-CustomerConfig-&lt;env&gt;


In [ ]:
import boto3
import json
import pandas as pd
from atsclientslibraries.customer_config import customerConfig

## Global Configs
#### Edit the following configuration for your particular AMC Instance.

In [ ]:
## The Team Name configured in the Data Lake platform
# This is the same value passed in the 'data_lake_parameters' team of ddk.json file
TEAM_NAME = "demoteam"

aws_region = "us-east-1"

ENV="dev"

aws_account_number = '<current aws account number>'

customerId = 'democustomer'

customerName = "DemoCustomer"

# The AMC Instance API Endpoint URL Can Be Found on Your AMC UI Page
amc_api_endpoint = "<AMC-API-ENDPOINT-URL>"


## Structure of Customer Configuration Records and their description. They are stored in CustomerConfig DynamoDB Table. 

```
{
    "customerId": <Customer ID which will be used to tag resources like bucket prefixes and table names in the platform. It has to unique and can be the name of the amc instance>,
    
    "customerName":<Customer Name. This will be used to idenify a customer. Keep it less than 5 characters and avoid any special characters>,
    
    "customerPrefix":<Prefix of Customer Name>
    
    "customer_hash_key":<Another unique field that can be used as an ID for the customer>
    
    "endemic_type": <ENDEMIC or NON-ENDEMIC; ENDEMIC: Brands that sell on Amazon, NONENDEMIC: Brands that do not sell on Amazon>,
    
    "amc":{
        "amcAccessCategory":<EXTERNAL or INTERNAL>,
        
        "amcApiEndpoint":<AMC instance endpoint. This can be requested or retrieved from the ticket raised to the AMC team >,
        
        "amcInstanceRegion":<Region of the AMC instance. This can be requested or retrieved from the ticket raised to the AMC team>,
        
        "amcGreenAwsAccount":<AMC Green Account. This can be requested or retrieved from the ticket raised to the AMC team>,
        
        "amcWorkflowPackages":<Customer ID>,
        
        "maximumConcurrentWorkflowExecutions":<Maximum number of workflows that can be executed concurrently by the amc instance. The limit is 10>,
        
         "wfm": {
             "amcWorkflowExecutionDLQSQSQueueName":<Name of the Dead Letter SQS Queue. Failed Executions are stored in this queue. Naming standard is
             {MicroserviceName}-{TeamName}-{PipelineName-{Environment}-workflowExecution-{CustomerID}-DLQ.fifo>,
             
             "amcWorkflowExecutionSQSQueueName":<Name of the AMC Workflow Execution Queue. 
                                                 Workflows that need to be executed are added to this queue. Naming standard is 
                                                 {MicroserviceName}-{TeamName}-{PipelineName}-{Environment-workflowExecution-{CustomerID}.fifo>,
             
             "enableWorkflowLibraryNewContent":<True or False; Set to True to allow customer to create new workflows in the library.>,
             
             "enableWorkflowLibraryRemoval":<True or False; Set to True to allow customer to delete workflows in the library.>,
             
             "enableWorkflowLibraryScheduleCreation":<True or False; Set to True to allow creating new workflow schedules.>,
             
             "enableWorkflowLibraryScheduleRemoval":<True or False; Set to True to allow deleting workflow schedules.>,
             
             "enableWorkflowLibraryUpdates":<True or False; Set to True to allow customer to update workflows.>,
             
             "snsTopicArn":<ARN of SNS Topic subscribed to WFM service. Naming standard is 
             arn:aws:sns:{AWSRegion}:{AWSAccountNumber}:{MicroserviceName}-{TeamName}-{PipelineName}-SNSTopic>
             
             "runWorkflowByCampaign":<dict containing parameters related to the Campaigns>
             
             "syncWorkflowStatuses": {
                        "amcWorkflowExecutionTrackingDynamoDBTableName": <Name of the table where execution tracking will be stored. Naming standard 
                        is <{MicroserviceName}-{TeamName}-{PipelineName}-AMCExecutionStatus>",
                        
                        "lastSyncedTime": "Auto-filled",
                        
                        "latestLastUpdatedTime": "Auto-filled",
                        
                        "workflowExecutionStatusLookBackHours": <Number of hours of execution that should be looked back to update the status>,
                        
                        "workflowStatusExpirationHours": <Number of hours after which the workflow status should be set to expired>,
                        
                        "workflowStatusExpirationTimeZone": <Time Zone in which the expiration should be calculated, for e.g America/New_York">,
                        
                        "WorkflowStatusRecordRetentionDays": <Number of Days the execution status records should be kept stored in the table>
              }
         }
    }
}
```

### Customer Config Records are created by CMS(Client Management Service). These records can be updated or deleted either directly through the console or using this notebook. Below is a sample customer config item. Use this sample item to create or update Customer Configurations.

In [ ]:
# customer configuration record (normally read from the the dynamodb table; format: {microservicename}-{teamname}-CustomerConfig-{env})
customer_config_record = {
  "AMC": {
    "amcAccessCategory": "EXTERNAL",
    "amcApiEndpoint": f"{amc_api_endpoint}",
    "amcInstanceRegion": f'{aws_region}',
    "amcWorkflowPackages": f"{customerId}",
    "maximumConcurrentWorkflowExecutions": 10,
    "WFM": {
      "amcWorkflowExecutionDLQSQSQueueName": f'wfm-{TEAM_NAME}--{ENV}-workflowExecution-{customerId}-DLQ.fifo',
      "amcWorkflowExecutionSQSQueueName": f'wfm-{TEAM_NAME}--{ENV}-workflowExecution-{customerId}.fifo',
      "enableWorkflowLibraryNewContent": True,
      "enableWorkflowLibraryRemoval": True,
      "enableWorkflowLibraryScheduleCreation": True,
      "enableWorkflowLibraryScheduleRemoval": True,
      "enableWorkflowLibraryUpdates": True,
      "runWorkflowByCampaign": {
        "campaignAttributionLagDays": 14,
        "campaignListDatabaseName": f'{TEAM_NAME}_amcdataset_dev_analytics',
        "campaignListTableName": f'{customerId}_active_campaigns_advertisers_v1_adhoc',
        "defaultWorkflowExecutionTimeZone": "America/New_York",
        "maximumCampaignAgeDays": 90,
        "maximumCampaignEndAgeDays": 18,
        "minimumCampaignAgeDays": 3
      },
      "snsTopicArn": f'arn:aws:sns:{aws_region}:{aws_account_number}:wfm-{TEAM_NAME}-SNSTopic-{ENV}',
      "syncWorkflowStatuses": {
        "amcWorkflowExecutionTrackingDynamoDBTableName": f'wfm-{TEAM_NAME}-AMCExecutionStatus-{ENV}',
        "lastSyncedTime": "2021-06-09T23:31:55",
        "latestLastUpdatedTime": "2021-06-09T22:54:24Z",
        "workflowExecutionStatusLookBackHours": 72,
        "workflowStatusExpirationHours": 72,
        "workflowStatusExpirationTimeZone": "America/New_York",
        "WorkflowStatusRecordRetentionDays": 90
      }
    }
  },
  "customer_hash_key": customerId,
  "customerId": customerId,
  "customerName": customerName,
  "customerPrefix": customerId,
  "endemicType": "ENDEMIC"
}

print(json.dumps(customer_config_record, indent=4))

### Create/Update a Customer Config Record:

In [ ]:
dynamodb_resp_wr = customerConfig.set_customer_config(customer_details=customer_config_record, TEAM_NAME=TEAM_NAME, ENV=ENV)
dynamodb_resp_wr

### Retrieve Current Customer Config Records:

In [ ]:
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option('max_rows', None)

df = customerConfig.get_customer_config(TEAM_NAME=TEAM_NAME, ENV=ENV)
df.head()

## Delete a Customer Config Record:

In [ ]:
dynamodb_resp_dl = customerConfig.delete_customer_config(customerId=customerId, TEAM_NAME=TEAM_NAME, ENV=ENV)
dynamodb_resp_dl